In [286]:
from keras.models import Sequential
from keras.layers import LSTM, Input
from keras.layers import Dense, Dropout,Conv1D,MaxPooling1D, Flatten
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import normalize
import pandas as pd
from keras.layers import RepeatVector, Bidirectional, GRU


In [287]:
merge_df = pd.read_csv('merged_data.csv')

In [109]:
merge_df.head()

,Unnamed: 0,frame_number,pedestrian_ID,pos_x,pos_y,v_x,v_y,Constant_speed,Speed_as_magnitude,euclidean_distance_1_destination,euclidean_distance_2_destination,euclidean_distance_3_destination,euclidean_distance_4_destination,euclidean_distance_with_obstacles,euclidean_distance_neighbor,euclidean_distance_neighbor_group
0,0,780.0,1.0,8.46,3.59,1.671714,0.176292,12,9,28.547123,15.453889,17.143920,6.938212,5.905144,0.0,100.000000
1,1,810.0,1.0,11.73,4.32,1.644127,0.544391,19,12,31.768974,18.809644,19.783212,3.597770,2.549283,0.0,100.000000
2,2,810.0,2.0,12.09,5.75,-1.589052,-0.065971,16,13,32.087941,19.524408,19.620513,3.025126,2.218885,0.0,0.000000
3,3,840.0,2.0,9.57,6.24,-1.207725,0.115437,8,11,29.573747,17.299866,17.081161,5.576444,4.652295,0.0,2.423523
4,4,840.0,3.0,11.94,6.77,-1.028251,0.307482,13,13,31.949280,19.701188,19.182262,3.390577,2.321748,0.0,2.423523


In [110]:
merge_df.columns

Index(['Unnamed: 0', 'frame_number', 'pedestrian_ID', 'pos_x', 'pos_y', 'v_x',
       'v_y', 'Constant_speed', 'Speed_as_magnitude',
       'euclidean_distance_1_destination', 'euclidean_distance_2_destination',
       'euclidean_distance_3_destination', 'euclidean_distance_4_destination',
       'euclidean_distance_with_obstacles', 'euclidean_distance_neighbor',
       'euclidean_distance_neighbor_group'],
      dtype='object')

In [111]:
merge_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [112]:
merge_df.head()

,frame_number,pedestrian_ID,pos_x,pos_y,v_x,v_y,Constant_speed,Speed_as_magnitude,euclidean_distance_1_destination,euclidean_distance_2_destination,euclidean_distance_3_destination,euclidean_distance_4_destination,euclidean_distance_with_obstacles,euclidean_distance_neighbor,euclidean_distance_neighbor_group
0,780.0,1.0,8.46,3.59,1.671714,0.176292,12,9,28.547123,15.453889,17.143920,6.938212,5.905144,0.0,100.000000
1,810.0,1.0,11.73,4.32,1.644127,0.544391,19,12,31.768974,18.809644,19.783212,3.597770,2.549283,0.0,100.000000
2,810.0,2.0,12.09,5.75,-1.589052,-0.065971,16,13,32.087941,19.524408,19.620513,3.025126,2.218885,0.0,0.000000
3,840.0,2.0,9.57,6.24,-1.207725,0.115437,8,11,29.573747,17.299866,17.081161,5.576444,4.652295,0.0,2.423523
4,840.0,3.0,11.94,6.77,-1.028251,0.307482,13,13,31.949280,19.701188,19.182262,3.390577,2.321748,0.0,2.423523


In [138]:
n_steps_in, n_steps_out = 8, 12 # number of input / number of outputs

In [139]:
# taking only posx and posy
def MakeInputOutput(dataset):
    # this method will make the base line model dataset which is basically x and y value itself.
    unique_ids = list(np.unique(dataset['pedestrian_ID']))
    X_values = dataset['pos_x']
    Y_values = dataset['pos_y']
    X_values = np.array(X_values).reshape((len(X_values), 1))
    Y_values = np.array(Y_values).reshape((len(Y_values), 1))
    stack = np.hstack((X_values,Y_values))
    return stack

In [140]:
# split a multivariate sequence into samples
import numpy as np
def split_sequences(sequences, n_steps_in, n_steps_out):
    # this function will make the sequence feature and output. in this method I am basically trying to convert time series into supervised structure where the input of first 8 sequence in time series would be the predicted 12 points
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [203]:
stack = MakeInputOutput(merge_df) # taking only X and y value for this moment

In [204]:
X, y = split_sequences(stack, n_steps_in, n_steps_out)

In [205]:
def addNewFeature(X_old, column, dataset):
    # this function i have made in order to add more feature and correspondly train and predict the model
    new_X = [] # current feature
    All_feature = [] # whole feature dataset
    #for i in columns:
    new_x = np.array(dataset[column])
    if sum(np.isnan(new_x))>0:
        new_x = np.nan_to_num(new_x)
    # converting to the n_setpes in features
    for i in range(len(new_x)):
        end_idx = i + n_steps_in
        seqx = new_x[i:end_idx]
        new_X.append(seqx)
    for i in range(len(X_old)):
        x = new_X[i].reshape(n_steps_in,1)
        new_feature_array   = np.append(X_old[i],x,axis=1)
        All_feature.append(new_feature_array)
    print("INFO: allnew features has been added")
    return np.array(All_feature)

In [206]:
merge_df.head()

,frame_number,pedestrian_ID,pos_x,pos_y,v_x,v_y,Constant_speed,Speed_as_magnitude,euclidean_distance_1_destination,euclidean_distance_2_destination,euclidean_distance_3_destination,euclidean_distance_4_destination,euclidean_distance_with_obstacles,euclidean_distance_neighbor,euclidean_distance_neighbor_group
0,780.0,1.0,8.46,3.59,1.671714,0.176292,12,9,28.547123,15.453889,17.143920,6.938212,5.905144,0.0,100.000000
1,810.0,1.0,11.73,4.32,1.644127,0.544391,19,12,31.768974,18.809644,19.783212,3.597770,2.549283,0.0,100.000000
2,810.0,2.0,12.09,5.75,-1.589052,-0.065971,16,13,32.087941,19.524408,19.620513,3.025126,2.218885,0.0,0.000000
3,840.0,2.0,9.57,6.24,-1.207725,0.115437,8,11,29.573747,17.299866,17.081161,5.576444,4.652295,0.0,2.423523
4,840.0,3.0,11.94,6.77,-1.028251,0.307482,13,13,31.949280,19.701188,19.182262,3.390577,2.321748,0.0,2.423523


In [207]:
merge_df.columns

Index(['frame_number', 'pedestrian_ID', 'pos_x', 'pos_y', 'v_x', 'v_y',
       'Constant_speed', 'Speed_as_magnitude',
       'euclidean_distance_1_destination', 'euclidean_distance_2_destination',
       'euclidean_distance_3_destination', 'euclidean_distance_4_destination',
       'euclidean_distance_with_obstacles', 'euclidean_distance_neighbor',
       'euclidean_distance_neighbor_group'],
      dtype='object')

In [208]:
features = ['v_x', 'v_y',
       'Constant_speed', 'Speed_as_magnitude',
       'euclidean_distance_1_destination', 'euclidean_distance_2_destination',
       'euclidean_distance_3_destination', 'euclidean_distance_4_destination',
       'euclidean_distance_with_obstacles', 'euclidean_distance_neighbor',
       'euclidean_distance_neighbor_group']

In [209]:
# here i am just adding all the features
for feature in features:
    new_X = addNewFeature(X,feature,merge_df)
    X = new_X

INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added


In [210]:
new_X.shape

(270, 8, 13)

In [211]:
merge_df.head()

,frame_number,pedestrian_ID,pos_x,pos_y,v_x,v_y,Constant_speed,Speed_as_magnitude,euclidean_distance_1_destination,euclidean_distance_2_destination,euclidean_distance_3_destination,euclidean_distance_4_destination,euclidean_distance_with_obstacles,euclidean_distance_neighbor,euclidean_distance_neighbor_group
0,780.0,1.0,8.46,3.59,1.671714,0.176292,12,9,28.547123,15.453889,17.143920,6.938212,5.905144,0.0,100.000000
1,810.0,1.0,11.73,4.32,1.644127,0.544391,19,12,31.768974,18.809644,19.783212,3.597770,2.549283,0.0,100.000000
2,810.0,2.0,12.09,5.75,-1.589052,-0.065971,16,13,32.087941,19.524408,19.620513,3.025126,2.218885,0.0,0.000000
3,840.0,2.0,9.57,6.24,-1.207725,0.115437,8,11,29.573747,17.299866,17.081161,5.576444,4.652295,0.0,2.423523
4,840.0,3.0,11.94,6.77,-1.028251,0.307482,13,13,31.949280,19.701188,19.182262,3.390577,2.321748,0.0,2.423523


In [213]:
X = normalize(X.reshape(X.shape[0], X.shape[1]*X.shape[2])).reshape(X.shape[0], X.shape[1],X.shape[2])

In [214]:
X_train,y_train = X[:int(len(X)*0.7)],y[:int(len(X)*0.7)]
X_valid,y_valid = X[int(len(X)*0.7):],y[int(len(X)*0.7):]

In [215]:
X_train.shape

(189, 8, 13)

In [219]:
X.shape, Y.shape

((270, 8, 13), (2, 12, 270))

In [232]:
# define CNN-lstm
def CNN_lstm_Model(num_feature):
    model = Sequential(name="CNN_lstm_Model")
    model.add(Conv1D(filters=48, kernel_size=3, activation='relu', input_shape=(n_steps_in, n_features))) # inputlayer
    # hidden layer
    model.add(Conv1D(filters=24, kernel_size=3, activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(MaxPooling1D(pool_size=2))
    # --------------------------------------------    
    model.add(Flatten())
    
    model.add(RepeatVector(n_steps_out))
    
    model.add(LSTM(200, activation='relu', return_sequences=True))
    #  output layer
    model.add(TimeDistributed(Dense(100, activation='relu')))
    model.add(TimeDistributed(Dense(2))) # 2 BECAUSE WE need to predict future posx and posy 
    model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics=['MeanSquaredError', 'MeanAbsoluteError','accuracy'], 
             )  
    return model
    

In [234]:
# fit model
if __name__ == "__main__":
    n_features = X_train.shape[-1]
    model = CNN_lstm_Model(n_features) 
    history2 = model.fit(X_train, y_train, epochs=100, verbose=1,  validation_data=(X_valid,y_valid))
    model.save('CNNLSTM_model')
    # writing the dictionary to the text file
    
    try:
        CNNLSTM_model = open('CNNLSTM_model.txt', 'wt')
        CNNLSTM_model.write(str(history2.history))
        CNNLSTM_model.close()
        print('CNNLSTM_model model summary has been created successfully')
    except:
        print("Unable to write to file")

Epoch 1/100
6/6 [==============================] - 4s 154ms/step - loss: 40.6676 - mean_squared_error: 40.6676 - mean_absolute_error: 5.6541 - accuracy: 0.4427 - val_loss: 50.4198 - val_mean_squared_error: 50.4198 - val_mean_absolute_error: 6.4399 - val_accuracy: 0.5309
Epoch 2/100
6/6 [==============================] - 0s 34ms/step - loss: 37.0994 - mean_squared_error: 37.0994 - mean_absolute_error: 5.3599 - accuracy: 0.5176 - val_loss: 41.9081 - val_mean_squared_error: 41.9081 - val_mean_absolute_error: 5.7584 - val_accuracy: 0.4805
Epoch 3/100
6/6 [==============================] - 0s 37ms/step - loss: 27.0095 - mean_squared_error: 27.0095 - mean_absolute_error: 4.3183 - accuracy: 0.5414 - val_loss: 36.7197 - val_mean_squared_error: 36.7197 - val_mean_absolute_error: 5.2990 - val_accuracy: 0.4805
Epoch 4/100
6/6 [==============================] - 0s 34ms/step - loss: 24.5435 - mean_squared_error: 24.5435 - mean_absolute_error: 4.1422 - accuracy: 0.5661 - val_loss: 31.6044 - val_mean

CNNLSTM_model model summary has been created successfully


In [273]:
features = ['v_x', 'v_y',
       'euclidean_distance_1_destination', 'euclidean_distance_2_destination',
       'euclidean_distance_3_destination', 'euclidean_distance_4_destination',]

In [276]:
X, y = split_sequences(stack, n_steps_in, n_steps_out)

In [277]:
# here i am just adding all the features
for feature in features:
    new_X = addNewFeature(X,feature,merge_df)
    X = new_X

INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added
INFO: allnew features has been added


In [278]:
X.shape

(270, 8, 8)

In [279]:
X = normalize(X.reshape(X.shape[0], X.shape[1]*X.shape[2])).reshape(X.shape[0], X.shape[1],X.shape[2])
X_train,y_train = X[:int(len(X)*0.7)],y[:int(len(X)*0.7)]
X_valid,y_valid = X[int(len(X)*0.7):],y[int(len(X)*0.7):]

In [280]:
# fit model
if __name__ == "__main__":
    n_features = X_train.shape[-1]
    model = CNN_lstm_Model(n_features) 
    history2 = model.fit(X_train, y_train, epochs=100, verbose=1,  validation_data=(X_valid,y_valid))
    model.save('CNNLSTM_model')
    # writing the dictionary to the text file
    
    try:
        CNNLSTM_model = open('CNNLSTM_model.txt', 'wt')
        CNNLSTM_model.write(str(history2.history))
        CNNLSTM_model.close()
        print('CNNLSTM_model model summary has been created successfully')
    except:
        print("Unable to write to file")

Epoch 1/100
6/6 [==============================] - 3s 132ms/step - loss: 40.3022 - mean_squared_error: 40.3022 - mean_absolute_error: 5.6239 - accuracy: 0.4392 - val_loss: 49.3732 - val_mean_squared_error: 49.3732 - val_mean_absolute_error: 6.3578 - val_accuracy: 0.5195
Epoch 2/100
6/6 [==============================] - 0s 39ms/step - loss: 34.5612 - mean_squared_error: 34.5612 - mean_absolute_error: 5.1265 - accuracy: 0.4347 - val_loss: 36.3196 - val_mean_squared_error: 36.3196 - val_mean_absolute_error: 5.2304 - val_accuracy: 0.5195
Epoch 3/100
6/6 [==============================] - 0s 34ms/step - loss: 24.4951 - mean_squared_error: 24.4951 - mean_absolute_error: 4.0717 - accuracy: 0.4563 - val_loss: 34.5707 - val_mean_squared_error: 34.5707 - val_mean_absolute_error: 5.1012 - val_accuracy: 0.4815
Epoch 4/100
6/6 [==============================] - 0s 36ms/step - loss: 21.9305 - mean_squared_error: 21.9304 - mean_absolute_error: 3.8475 - accuracy: 0.5653 - val_loss: 26.8365 - val_mean

INFO:tensorflow:Assets written to: CNNLSTM_model\assets


CNNLSTM_model model summary has been created successfully


In [281]:
def buildmodel(x_train,y_train):
    # Create the SVR regressor
    n_sample,nx, ny = X_train.shape
    trainx = X_train.reshape((n_sample, nx*ny))
    
    n_sample,nx, ny = y_train.shape
    trainy = y_train.reshape((n_sample, nx*ny))
    
    svr = SVR(epsilon=0.2)
    # Create the Multioutput Regressor
    mor = MultiOutputRegressor(svr)
    
    # Train the regressor
    mor = mor.fit(trainx, trainy)
    
    return mor


def svmpredict(model, x):
    m,n = x.shape
    x = x.reshape(1,m*n)
    y_hat = model.predict(x)
    return y_hat
# fit model
if __name__ == "__main__":
   mor = buildmodel(X_train,y_train )
   pred = svmpredict(mor, X[0]).reshape(12,2)
   # saving the model
   model= 'svm_multiout.sav'
   pickle.dump(mor,open(model,'wb'))
   print('model saved successfully')


model saved successfully


In [284]:
def buildModel(X_train):
    ##### Step 3 - Specify the structure of a Neural Network
    model = Sequential(name="GRU_Model") # Model
    model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
    model.add(Bidirectional(GRU(units=64, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-GRU-Encoder-Layer')) # Encoder Layer
    model.add(RepeatVector(n_steps_out, name='Repeat-Vector-Layer')) # Repeat Vector
    model.add(Bidirectional(GRU(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-GRU-Decoder-Layer')) # Decoder Layer
    #ouput layer
    model.add(TimeDistributed(Dense(units=2, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x

    model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['MeanSquaredError', 'MeanAbsoluteError','accuracy'], 
             )
    
    return model
    

def predict(model, x, n_features):
    x = x.reshape(1,8,n_features)
    y_hat = model.predict(x)
    print(y_hat) 

In [288]:
model = buildModel(X_train) 
history2 = model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_valid,y_valid))
model.save('GRU_model')

Epoch 1/100
6/6 [==============================] - 13s 510ms/step - loss: 39.1387 - mean_squared_error: 39.1387 - mean_absolute_error: 5.5341 - accuracy: 0.5494 - val_loss: 46.1110 - val_mean_squared_error: 46.1110 - val_mean_absolute_error: 6.1182 - val_accuracy: 0.4805
Epoch 2/100
6/6 [==============================] - 0s 34ms/step - loss: 33.6056 - mean_squared_error: 33.6056 - mean_absolute_error: 5.0790 - accuracy: 0.5661 - val_loss: 37.5198 - val_mean_squared_error: 37.5198 - val_mean_absolute_error: 5.4363 - val_accuracy: 0.4805
Epoch 3/100
6/6 [==============================] - 0s 34ms/step - loss: 24.9067 - mean_squared_error: 24.9067 - mean_absolute_error: 4.2882 - accuracy: 0.5661 - val_loss: 22.9797 - val_mean_squared_error: 22.9797 - val_mean_absolute_error: 4.0744 - val_accuracy: 0.4805
Epoch 4/100
6/6 [==============================] - 0s 35ms/step - loss: 14.0363 - mean_squared_error: 14.0363 - mean_absolute_error: 3.0757 - accuracy: 0.5661 - val_loss: 10.3654 - val_mea

INFO:tensorflow:Assets written to: GRU_model\assets


INFO:tensorflow:Assets written to: GRU_model\assets


In [291]:
try:
    GRU_model = open('GRU_model.txt', 'wt')
    GRU_model.write(str(history2.history))
    GRU_model.close()
    print('GRU_model model summary has been created successfully')
except:
    print("Unable to write to file")


GRU_model model summary has been created successfully


In [292]:
def buildModel():
    model = Sequential(name="lstm_model")
    model.add(LSTM(400, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(200, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(2))) # output layer
    #model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps_in, n_features)))
    #model.add(Dense(n_features))
    model.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['MeanSquaredError', 'MeanAbsoluteError','accuracy'], 
              loss_weights=None, 
              weighted_metrics=None, 
              run_eagerly=None, 
              steps_per_execution=None 
             )
    return model


In [293]:
model = buildModel() 
history1 = model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_valid,y_valid))
model.save('MultiEndec_model')

Epoch 1/100
6/6 [==============================] - 5s 240ms/step - loss: 40.4116 - mean_squared_error: 40.4116 - mean_absolute_error: 5.6367 - accuracy: 0.5661 - val_loss: 48.7854 - val_mean_squared_error: 48.7854 - val_mean_absolute_error: 6.3256 - val_accuracy: 0.4805
Epoch 2/100
6/6 [==============================] - 1s 96ms/step - loss: 33.5678 - mean_squared_error: 33.5678 - mean_absolute_error: 4.9794 - accuracy: 0.5661 - val_loss: 27.5518 - val_mean_squared_error: 27.5518 - val_mean_absolute_error: 4.4434 - val_accuracy: 0.4805
Epoch 3/100
6/6 [==============================] - 1s 91ms/step - loss: 26.5529 - mean_squared_error: 26.5529 - mean_absolute_error: 4.4308 - accuracy: 0.5661 - val_loss: 38.0805 - val_mean_squared_error: 38.0805 - val_mean_absolute_error: 5.4718 - val_accuracy: 0.4805
Epoch 4/100
6/6 [==============================] - 1s 100ms/step - loss: 28.9884 - mean_squared_error: 28.9884 - mean_absolute_error: 4.6652 - accuracy: 0.5661 - val_loss: 34.3905 - val_mea

INFO:tensorflow:Assets written to: MultiEndec_model\assets


In [296]:
try:
    MultiEndec_model = open('MultiEndec_model.txt', 'wt')
    MultiEndec_model.write(str(history1.history))
    MultiEndec_model.close()
    print('MultiEndec_model  summary has been created successfully')
except:
    print("Unable to write to file")


MultiEndec_model  summary has been created successfully


Conclusion : LSTM multiencoder : val_accuracy: 0.5144 and mean_squared_error: 8.1263
             CNN LSTM model    : val_accuracy: 0.5556 and mean_squared_error: 8.2511
             GRU seuence Model : val_accuracy: 0.4805 and mean_squared_error: 9.7947 
             
Previous Case where we have trained with 3 features POSX, POSY and Constant Speed We have seen that 
lstm model accuracy decrease while cnn lstm model accuracy has slightly increased after adding the feature.

CNN_LSTM model is performing in both test cases slightly better.